In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv('WeatherData.csv')
data.iloc[:7]

,Date,Time,Temperature,Dew Point,Humidity,Wind,Wind Speed,Wind Gust,Pressure,Precip.,Condition
0,2022-01-01,1:00 AM,43 °F,41 °F,93 °%,W,5 °mph,0 °mph,30.09 °in,0.0 °in,Fair
1,2022-01-01,1:30 AM,45 °F,41 °F,87 °%,NW,3 °mph,0 °mph,30.09 °in,0.0 °in,Fair
2,2022-01-01,2:00 AM,43 °F,41 °F,93 °%,WNW,3 °mph,0 °mph,30.09 °in,0.0 °in,Fair
3,2022-01-01,2:30 AM,45 °F,41 °F,87 °%,WNW,3 °mph,0 °mph,30.09 °in,0.0 °in,Fair
4,2022-01-01,3:00 AM,43 °F,41 °F,93 °%,WNW,6 °mph,0 °mph,30.09 °in,0.0 °in,Fair
5,2022-01-01,3:30 AM,43 °F,39 °F,87 °%,WNW,3 °mph,0 °mph,30.09 °in,0.0 °in,Fair
6,2022-01-01,4:00 AM,41 °F,37 °F,87 °%,WNW,3 °mph,0 °mph,30.09 °in,0.0 °in,Fair


In [2]:
data = data.drop(columns = ["Time", "Dew Point", "Wind", "Wind Speed", "Wind Gust", "Pressure", "Precip."])
data["Temperature"] = data["Temperature"].apply( lambda x: round((int(x[:2])-32) * 5/9 + 273, 1) ) #En Kelvins
data["Humidity"] = data["Humidity"].apply(lambda x: int(x[:2]))
data['Date'] = pd.to_datetime(data['Date']) #Esto convierte la columna Date a un formato datetime, lo que permite utilizar métodos como .dt.month
data['Month'] = data['Date'].dt.month #El método .dt.month extrae el número del mes de la fecha.
data = data.drop(columns = ["Date"])
labels = data.Condition.unique()
labels

array(['Fair', 'Mostly Cloudy', 'Mist', 'Fog', 'Fair / Windy',
       'Partly Cloudy', 'Light Rain', 'Light Drizzle', 'Drizzle', 'Rain',
       'Cloudy', 'Rain Shower', 'Heavy Rain Shower', 'Light Rain Shower',
       'Light Rain / Windy', 'Mostly Cloudy / Windy',
       'Partly Cloudy / Windy', 'Cloudy / Windy', 'Rain / Windy',
       'Light Rain Shower / Windy', 'Rain Shower / Windy', 'Smoke',
       'Heavy Rain', 'Thunder in the Vicinity', 'Thunder',
       'Light Rain with Thunder', 'T-Storm', 'Heavy T-Storm',
       'Heavy T-Storm / Windy', 'Patches of Fog'], dtype=object)

In [3]:
processed_data = data.copy()

# norain = ['Mostly Cloudy', 'Fair', 'Partly Cloudy', 'Haze', 'Fog', 'Mostly Cloudy / Windy', 'Cloudy', 'Partly Cloudy / Windy']
# mightrain = ['Showers in the Vicinity', 'Thunder in the Vicinity', 'Thunder']
# light_rain =  ['Light Rain Shower', 'Light Rain', 'Light Rain / Windy']
# rain = ['Rain Shower', 'Rain', 'Heavy Rain', 'Heavy Rain Shower']
# storm = ['Light Rain with Thunder', 'Heavy T-Storm', 'T-Storm']
norain = ['Mostly Cloudy', 'Fair', 'Partly Cloudy', 'Haze', 'Fog', 'Mostly Cloudy / Windy', 'Cloudy', 'Partly Cloudy / Windy']
mightrain = ['Showers in the Vicinity', 'Thunder in the Vicinity', 'Thunder']
rain = ['Rain Shower', 'Light Rain Shower', 'Light Rain', 'Rain', 'Heavy Rain', 'Light Rain with Thunder', 'Heavy Rain Shower', 'Heavy T-Storm', 'T-Storm', 'Light Rain / Windy']


processed_data["NoRain"] = 0
processed_data["MightRain"] = 0
# processed_data["LightRain"] = 0
processed_data["Rain"] = 0
# processed_data["Storm"] = 0

processed_data["NoRain"].loc[data.Condition.isin(norain)] = 1
processed_data["MightRain"].loc[data.Condition.isin(mightrain)] = 1
# processed_data["LightRain"].loc[data.Condition.isin(light_rain)] = 1
processed_data["Rain"].loc[data.Condition.isin(rain)] = 1
# processed_data["Storm"].loc[data.Condition.isin(storm)] = 1
processed_data = processed_data.drop(columns="Condition")

processed_data

C:\Users\javic\AppData\Local\Temp\ipykernel_31668\1567509972.py:19: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  processed_data["NoRain"].loc[data.Condition.isin(norain)] = 1
C:\Users\javic\AppData\Local\Temp\ipykernel_31668\1567509972.py:1

,Temperature,Humidity,Month,NoRain,MightRain,Rain
0,279.1,93,1,1,0,0
1,280.2,87,1,1,0,0
2,279.1,93,1,1,0,0
3,280.2,87,1,1,0,0
4,279.1,93,1,1,0,0
...,...,...,...,...,...,...
33239,280.8,66,1,1,0,0
33240,280.2,71,1,1,0,0
33241,280.2,71,1,1,0,0
33242,279.1,70,1,1,0,0


In [4]:
# y = processed_data[["NoRain", "MightRain","LightRain", "Rain", "Storm"]].to_numpy()[7:]
# rawx = processed_data.drop(columns = ["NoRain", "MightRain","LightRain", "Rain", "Storm"]).to_numpy()
y = processed_data[["NoRain", "MightRain", "Rain"]].to_numpy()[7:]
rawx = processed_data.drop(columns = ["NoRain", "MightRain", "Rain"]).to_numpy()
x = []
temp = np.array(0)
for i in range(len(rawx)-7):
    temp = rawx[i:i+6].flatten()
    #print(temp)
    x.append(temp)
x = np.array(x)
x.shape, y.shape

((33237, 18), (33237, 3))

In [18]:
print(x[1])
print(y[1])

[280.2  87.    1.  279.1  93.    1.  280.2  87.    1.  279.1  93.    1.
 279.1  87.    1.  278.   87.    1. ]
[1 0 0]


In [5]:
import tensorflow as tf
import keras
from keras import layers
from sklearn.model_selection import train_test_split

xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.20, random_state=33, stratify=y)
#Para asegurar que haya datos de prueba para todas las clases al usar train_test_split de scikit-learn, debes hacer que la partición sea estratificada según las clases de la variable objetivo y. Esto se logra utilizando el parámetro stratify.

In [6]:

model = tf.keras.Sequential()
model.add(layers.Dense(14, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(8, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(3, activation='softmax'))
optimizer = tf.optimizers.Adam(learning_rate=0.0001)

model.compile(loss = 'categorical_crossentropy',
              optimizer = optimizer,
              metrics=['accuracy'])

model.fit(xtrain, ytrain, epochs=40, validation_split=0.1, batch_size=128)

test_results = model.evaluate(xtest, ytest, verbose=1)

model.export('model')

Epoch 1/40
187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6684 - loss: 25.9448 - val_accuracy: 0.9650 - val_loss: 4.6334
Epoch 2/40
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 925us/step - accuracy: 0.7372 - loss: 15.2496 - val_accuracy: 0.9650 - val_loss: 3.9761
Epoch 3/40
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7869 - loss: 10.0742 - val_accuracy: 0.9650 - val_loss: 3.1318
Epoch 4/40
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 991us/step - accuracy: 0.8094 - loss: 7.0799 - val_accuracy: 0.9650 - val_loss: 2.3041
Epoch 5/40
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 920us/step - accuracy: 0.8278 - loss: 4.9000 - val_accuracy: 0.9650 - val_loss: 1.5485
Epoch 6/40
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 922us/step - accuracy: 0.8386 - loss: 3.2131 - val_accuracy: 0.9650 - val_loss: 0.9636
Epoch 7/40
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 917us/step - accuracy: 0.8484 - loss: 2.1877 - val_accuracy: 0.9646 - val_loss: 0.4529
Epoch 8/40
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 920us/step - accuracy: 0.8635 - loss: 1.1741 - v

In [21]:
import tensorflow as tf
# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model("model/") # path to the SavedModel directory
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)


In [23]:

import os
model_h_size = os.path.getsize("model.tflite")
print(f"Header file, model.h, is {model_h_size:,} bytes.")

Header file, model.h, is 3,728 bytes.


In [24]:
print(model.input_shape)

(None, 18)


In [26]:
x_new = np.array([
    [285, 88, 3, 285, 88, 3, 285, 88, 3, 285, 88, 3, 285, 88, 3, 285, 88, 3],
    # Aquí añade más muestras de prueba si lo deseas
])

# Realizar predicciones
predictions = model.predict(x_new)

# Obtener las clases predichas (índice del valor más alto en las predicciones)
predicted_classes = np.argmax(predictions, axis=1)

# Mostrar los resultados
print("Predicciones:", predicted_classes)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Predicciones: [0]
